In [3]:
import time
from matplotlib import pyplot as plt
import numpy as np
import mxnet as mx
import pandas as pd
from mxnet import autograd, gluon
import gluoncv as gcv
from gluoncv.utils import viz
from gluoncv.data import VOCDetection
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.ssd import SSDDefaultTrainTransform
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from test_trained import get_dataloader
from finetune import VOCLike

## Define AP retrieval
This is a hacky way to return the AP of a single image by using magic methods from VOC07MApMetric

In [15]:
def get_aps(metric_inst):
    aps = []
    recall, precs = metric_inst._recall_prec()
    for l, rec, prec in zip(range(len(precs)), recall, precs):
        ap = metric_inst._average_precision(rec, prec)
        aps.append(ap)
    return aps

## Do all the preliminaries
Define class names, load the network with the trained weights, and instantiate the validation routine

In [4]:
# declare the classes
classes = ['brittle_star', 'cnidaria', 'eel', 'misc_fish', 'mollusc',
           'orange_roughy_edge', 'orange_roughy', 'sea_anemone', 'sea_feather',
           'sea_star','sea_urchin']

# load the model
net = gcv.model_zoo.get_model('ssd_512_mobilenet1.0_custom', classes=classes,
                              pretrained_base=False)

# load the weights
net_wght = '/home/ec2-user/SageMaker/csiro-aos-object-detection/clf-outputs/\
010221-ssd_512_mobilenet1.0_voc_roughy_OPall_10.params'

net.load_parameters(net_wght)

# put on the GPU [just run on CPU for now - ECO 020721]
#contx = [mx.gpu(0)]
#print('GPU found')
# net.collect_params().reset_ctx(contx)
ctx = [mx.cpu(0)]

# load the network 
test_set = '/home/ec2-user/SageMaker/csiro-aos-object-detection/VOCportALL/'
test_run = 'OP19/test'
test_dataset = VOCLike(root=test_set, splits=(('OP', f'{test_run}'),))

# instantiate the validation class
val_metric = VOC07MApMetric(iou_thresh=0.5, class_names=test_dataset.classes)

# get the test data
test_data = get_dataloader(net, test_dataset, 512, 16, 0, 'ctx')

## Get the AP for each image
Loop over each batch of data and retrieve predicted scores and boudning boxes with the associated ground truth. Then step through each image in the batch to return the AP score of each class in that image. Save the resulting listing of scores to an output list. The result should be the same length as the image list defined above.

In [48]:
# loop over each batch in the test data
aps = []
flag = 1
for batch in test_data:
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
    det_bboxes = []
    det_ids = []
    det_scores = []
    gt_bboxes = []
    gt_ids = []
    gt_difficults = []
    #print(batch[0].shape)
    for x, y in zip(data, label):
        # get prediction results
        ids, scores, bboxes = net(x)
        det_ids.append(ids)
        det_scores.append(scores)
        # clip to image size
        det_bboxes.append(bboxes.clip(0, batch[0].shape[2]))
        # split ground truths
        gt_ids.append(y.slice_axis(axis=-1, begin=4, end=5))
        gt_bboxes.append(y.slice_axis(axis=-1, begin=0, end=4))
        gt_difficults.append(y.slice_axis(axis=-1, begin=5, end=6) if y.shape[-1] > 5 else None)
        
    # loop over each image to return the AP of each class
    tmp = []
    #print(det_ids[0].shape[0])
    for ii in range(1,det_ids[0].shape[0]+1):
        ind = ii-1
        # reset the metric class
        val_metric.reset()
        val_metric.update([det_bboxes[0][ind:ii,:,:]], [det_ids[0][ind:ii,:,:]], [det_scores[0][ind:ii,:,:]], 
                  [gt_bboxes[0][ind:ii,:,:]], [gt_ids[0][ind:ii,:,:]], [gt_difficults[0][ind:ii,:,:]])
        tmp.append(get_aps(val_metric))
    aps.extend(tmp)
    print('done with itr', str(flag))
    flag+=1

done with itr 1
done with itr 2
done with itr 3
done with itr 4
done with itr 5
done with itr 6
done with itr 7
done with itr 8
done with itr 9
done with itr 10
done with itr 11
done with itr 12
done with itr 13


## Data wrangle
Associate the list of AP scores with the image ID

In [56]:
# load the list of image IDs [should be same as above]
ptf = os.path.join(test_set, 'OP','ImageSets','Main',f'{test_run}.txt')
with open(ptf, 'r') as ff:
    img_ids = list(ff)
    ff.close()

In [60]:
df = pd.DataFrame(aps, index=img_ids, columns=classes)

In [61]:
df

,brittle_star,cnidaria,eel,misc_fish,mollusc,orange_roughy_edge,orange_roughy,sea_anemone,sea_feather,sea_star,sea_urchin
20190715-014739-285\n,NaN,NaN,NaN,NaN,NaN,NaN,0.579650,NaN,NaN,NaN,NaN
20190715-015105-227\n,NaN,NaN,NaN,NaN,NaN,0.848485,1.000000,NaN,NaN,NaN,NaN
20190715-015017-230\n,NaN,NaN,NaN,NaN,NaN,0.375000,0.672900,NaN,NaN,NaN,NaN
20190715-014721-285\n,NaN,NaN,NaN,NaN,NaN,0.333333,1.000000,0.0,NaN,NaN,NaN
20190715-014647-237\n,NaN,NaN,NaN,NaN,NaN,NaN,0.974026,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20190715-014609-287\n,NaN,NaN,NaN,0.0,NaN,NaN,0.848485,NaN,NaN,NaN,NaN
20190715-015219-249\n,NaN,NaN,NaN,NaN,NaN,0.602273,0.877005,NaN,NaN,NaN,NaN
20190715-014719-259\n,NaN,NaN,NaN,NaN,NaN,0.125000,1.000000,NaN,NaN,NaN,NaN
20190715-014717-234\n,NaN,NaN,NaN,NaN,NaN,NaN,0.948052,NaN,NaN,NaN,NaN
